In [1]:
import numpy as np

In [6]:
from bayesian.bbn import build_bbn
from bayesian.utils import make_key

def f_burglary(burglary):
    if burglary:
        return 0.01
    else:
        return 0.99

def f_earthquake(earthquake):
    if earthquake:
        return 10e-6
    else:
        return 1.-10e-6

def f_alarm(burglary, earthquake, alarm):
    table = dict()
    table['fff'] = 0.999
    table['fft'] = 0.001
    table['ftf'] = 0.59
    table['ftt'] = 0.41
    table['tff'] = 0.05
    table['tft'] = 0.95
    table['ttf'] = 0.02
    table['ttt'] = 0.98
    return table[make_key(burglary, earthquake, alarm)]

def f_radio(earthquake, radio):
    table = dict()
    table['ff'] = 1.0
    table['ft'] = 0.0
    table['tf'] = 0.0
    table['tt'] = 1.0
    return table[make_key(earthquake, radio)]


g = build_bbn(
    f_burglary,
    f_earthquake,
    f_alarm,
    f_radio)
g.q()
g.q(alarm=1,radio=1)
g.q(alarm=1)

+------------+-------+----------+
| Node       | Value | Marginal |
+------------+-------+----------+
| alarm      | False | 0.989506 |
| alarm      | True  | 0.010494 |
| burglary   | False | 0.990000 |
| burglary   | True  | 0.010000 |
| earthquake | False | 0.999990 |
| earthquake | True  | 0.000010 |
| radio      | False | 0.999990 |
| radio      | True  | 0.000010 |
+------------+-------+----------+
+------------+-------+----------+
| Node       | Value | Marginal |
+------------+-------+----------+
| alarm      | False | 0.000000 |
| alarm*     | True* | 1.000000 |
| burglary   | False | 0.976425 |
| burglary   | True  | 0.023575 |
| earthquake | False | 0.000000 |
| earthquake | True  | 1.000000 |
| radio      | False | 0.000000 |
| radio*     | True* | 1.000000 |
+------------+-------+----------+
+------------+-------+----------+
| Node       | Value | Marginal |
+------------+-------+----------+
| alarm      | False | 0.000000 |
| alarm*     | True* | 1.000000 |
| burglary   |